## Scenario

The input file is around 2GB (~300 million lines), and there is just one commidity computer available(500MB free memory, 10GB disk space).

## Approaches

If the output unique IDs could fit in the memory, this problem is easy to solve by read the input in a small batch, then store the unique IDs in a `set`, and write the this set to a file after processing all the input IDs.

However, this problem could be solved from different perspectives.
- Linux Tools
- Writing a Program
- Database

### Approach I: Linux's **sort** and **uniq**

The `sort` could do a [external sort](https://en.wikipedia.org/wiki/External_sorting) when the file is too large, and then `uniq` can get all the unique values.

Here is the bash command.
```bash
$sort input_id.txt | uniq > uniq_id_bash.txtash
```

### Approach II: Devide and Conquer (Python Program)

**Algorithm**  
Go through the input file multiple times, count the unique IDs starting with a specific character (such as 'a') each time. After done, append these unique IDs in a the output file, which is empty in the beginning.

and then move to another one (such as 'b' etc.). 

**Example**  

```
xNDGN3R
8guP0Af
VHgwqgA
Wy0AXoo
wy0AXoo
xNDGN3R
toS3f6T
8bIgIXy
xNDGN3R
```

Iteration 1: Read the file line by line, and collect all the uqique IDs start with 'a': None. Do nothing since there is no unique ID starting with 'a'.
...
Iteration 20:Read the file line by line again,and collect all the uqique IDs start with 't': ('toS3f6T'). So, append it to the output (could be a file on disk).
...
Iteration... all the uqique IDs start with 'x': ('xNDGN3R'). So, append it to the output file.
Iteration 61: Read the file line by line, and collect all the uqique IDs start with '8': ('8bIgIXy', '8guP0Af'), and append it to the output.
Iteration 62: Read the file line by line, and collect all the uqique IDs start with '9': None

After 62 iterations, the output would be:
```
toS3f6T
xNDGN3R
VHgwqgA
Wy0AXoo
8bIgIXy
8guP0Af
```

**Implemention**

Here is the python program.

In [ ]:
# python 3.6+

import os
import fileinput
import collections

from datetime import datetime

CHAR_SET = 'abcdefghijklmnopqrstuvwxyz' + 'ABCDEFGHIJKLMNOPQRSTUVWXYZ' + '0123456789'

INPUT_FILE = 'input_id.txt' # default input file
OUTPUT_FILE = 'uniq_id.txt' # default output file

#Return a set of IDs started with `start_char` in file `file
def extract_uniq_ids(input_file, output_file):
    uniq_ids = set()
    
    for char in CHAR_SET:
        uniq_ids.clear()
        
        start = datetime.now()
        with fileinput.input(files = input_file) as f:
            for line in f: # cannot use f.readlines() since the file is too large
                line = line.strip()
                if line.startswith(char) and line not in uniq_ids:
                    uniq_ids.add(line)

        if uniq_ids:
            with open(output_file, 'a') as f:
                f.write('\n'.join(uniq_ids))
                f.write('\n')

        end = datetime.now()
        print(f'Extracted unique IDs starting with {char} using {end - start}.')
    return uniq_ids

if __name__ == '__main__':
    extract_uniq_ids(INPUT_FILE, OUTPUT_FILE)

**Limitation**

The success of the above algorithm replys on the volume of unique IDs starting with each character. If the data is highly skewed, certain initial char has so many unique IDs that could not be fit in the memoery, this program will fail due to out of memory issue. Developing other rules to 'partition' the data properly could fix this issue, such as taking the first and last char, or use string's hash value mod N (N is the times of the space/size of unique IDs / available memeory).

Note:
Sampling data could get a rough skewness reference, and `shuf -n x file` in Linux can get a random sample of x lines.

**Complexity and Optimization**

The complexity is linear $O(N)$ because of scanning the big file 62 times, while 'N' is the total number of IDs in the input file. There is no way to reduce the complexity dramtically because you have to go through all the N IDs to find out all the unique IDs.

However, if the number of unique IDs(M) is way less than the total IDs(N), we could process the huge file in batch mode and then combine these IDs with privious ID set. The algorithm is decribed as below:

1. Read 10 million lines each time, and collect the unique IDs grouped by the initial character.
2. Save each group to a file, such as id-a.txt, id-b.txt, etc.
3. Continue to process the next 10 million lines, and merge the group to previous group in the file.
4. Proceed until it reaches the end of the file.
5. Then append these small files to a new file to get all the unique IDs of the input file.

In this way, the complexity could be reduced to $O(M*S)$, while 'M' is the number of unique IDs, and 'S' is the number of splits. (If you have 300 millions and process 10 million each time, the S is 30.) In this way, the huge file could only be read once, while the trade off is the small files will be read multiple times.

Here is the optimized implementation in case of only a relative small unique IDs volume.

In [6]:
PREFIX = 'id-'
BATCH_NUM = 10000000
INPUT_FILE = 'input_id.txt'

import os.path
import fileinput

from collections import defaultdict

def merge_ids(file, other_ids):
    ids = set()
    
    if os.path.isfile(file): 
        with open(file, 'r') as f:
            for line in f:
                line = line.strip()
                if line and line not in ids:
                    ids.add(line)
            f.close()

    ids = ids.union(other_ids)
    
    with open(file, 'w') as f:
        f.write('\n'.join(ids))
        f.write('\n')
        f.close()

    return True

def extract_ids_batch(input_file, prefix = PREFIX, batch = BATCH_NUM):
    m = defaultdict(set) # k:'[a-z|A-Z|0-9]', value = set()
    with fileinput.input(files = input_file) as f:
        while True:
            m.clear()
            for _ in range(BATCH_NUM):
                line = f.readline()
                if line and line != '\n':
                    m[line[:1]].add(line.strip())
                else:top
                    
                    break

            if not m:break
            
            for start_char, ids in m.items():
                if start_char.isupper(): start_char += '_' # bugfix for case insensitive OS like mac
                
                file_to_merge = prefix + start_char + '.txt'
                merge_ids(file_to_merge, ids)

if __name__ == '__main__':
    extract_ids_batch(INPUT_FILE)

## Approach III: Use database column's 'unique' feature

**Intution**
1. Take the same step as before to split the files into small files.
2. Create a table `unique_id` with one column `id` which is unique in MySQL.
3. Take one files and read all the IDs, then batch insert these values into the database using `ignore`.
4. Continue to process all the splitted files till the end.

The reason why it works is because the duplicated values are ignored. Here is the SQL code for reference.

```sql
create table if not EXISTS unique_id (id char(7) UNIQUE not NULL);
load data infile '/var/lib/mysql/input_id.txt' ignore into table unique_id;
```

**Verification**



## Appendix

### Data Generation

The goal is to generate a about 2GB size files having IDs (a-z|A-Z|0-9), the unique IDs size of which is above 500MB.

Quick estimation:
- The lenght of a single ID is 7B.
- The total ID (lines) in the generated file should be more than 300 millions.
- The unique ID (lines) should be more than 70 millions.

In [1]:
### Prepare a input ID file for demostration purpose: input_id.txt

import random

def gen_ids(valid_chars, length, N):
    """Generate N number of id:
    - each char of the id is in valid_chars
    - the lenght of id is length
    """
    ids = []
    for n in range(N):
        id = random.choices(valid_chars, k=length)
        ids.append(''.join(id))
    return ids


CHAR_SET = 'abcdefghijklmnopqrstuvwxyz' + 'ABCDEFGHIJKLMNOPQRSTUVWXYZ' + '0123456789'
INPUT_FILE = 'input_id.txt' # file for generated IDs
ID_LENGTH = 7
N = 30000000 # 30 millions, so run this for 6 times

with open(INPUT_FILE, 'a') as f:
    f.write('\n'.join(gen_ids(CHAR_SET, ID_LENGTH, N)))

### Data Validation

Use this program to validate the format of IDs.

In [31]:
# validate the IDs

import os
import fileinput

def validate_id(path): # path is either file or dir
    if os.path.isfile(path):
        with fileinput.input(files=path) as f:
            for line in f:
                line = line.strip()
                if not line: continue
                if len(line) != ID_LENGTH:
                    print(line)
                    return False
                for c in line:
                    if not c in CHARS_SET:
                        print(line)
                        return False
    else:
        for root, dirs, files in os.walk(path):
            for file in files:
                file = os.sep.join((root, file))
                validate_id(file)
    return True

if '__name__' == '__main__':
    validate_id(INPUT_FILE)

### Effenciency Comparation

**Input Data**  
File Name: input_id.txt  
Size:      2.0GB  
Lines:     274 millions  

**Output Date**  
File Name: uniq_id.txt  
Size:      540MB  
Lines:     ~68 millions  

**Running Machine Profile**  
Hardware: AWS Lightsail (1 GB RAM, 1 vCPU, 40 GB SSD)  
Software: Ubuntu 16.04.5 LTS (GNU/Linux 4.4.0-1074-aws x86_64)  

*Available memory is around 400MB after startup*  


**Approaches**  

| Approach                | Running Time  | Note                                  |
| ----------------------- | ------------- | ------------------------------------- |
| Linux `sort` and `uniq` | < 20 min      | Most effecient                        |
| Programming             | ~ 150 min     | ~2.5 min/each                         |
| Programming (Enhanced)  |               |                                       |
| Database (MySQL)        | > 60 min      | terminated mannuall due to 68 mln rows|

Note:  
- In the programming approach, the performace degrade to 1/10 after around 'k' (inital char), so suggest to output to each individual files rather than appending to a huge file.  
- 